In [1]:
from subprocess import check_output
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from tqdm import tqdm

import torch.backends.cudnn as cudnn
import random

## Claim: the data preprocessing part is from https://www.kaggle.com/code/pavansanagapati/a-simple-cnn-model-beginner-guide/notebook

In [2]:
# This code is from https://www.kaggle.com/code/pankajj/fashion-mnist-with-pytorch-93-accuracy

class Data(Dataset):
    
    def __init__(self, data, transform = None):
        self.fashion_MNIST = list(data.values)
        self.transform = transform
        
        label = []
        image = []
        
        for i in self.fashion_MNIST:
             # first column is of labels.
            label.append(i[0])
            image.append(i[1:])
        self.labels = np.asarray(label)
        # Dimension of Images = 28 * 28 * 1. where height = width = 28 and color_channels = 1.
        self.images = np.asarray(image).reshape(-1, 28, 28, 1).astype('float32')

    def __getitem__(self, index):
        label = self.labels[index]
        image = self.images[index]
        
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.images)

In [3]:
train_csv = pd.read_csv("fashion-mnist_train.csv")
test_csv = pd.read_csv("fashion-mnist_test.csv")

train_data = Data(train_csv, transform=transforms.Compose([transforms.ToTensor()]))
test_data = Data(test_csv, transform=transforms.Compose([transforms.ToTensor()]))

batch_size = 256
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [4]:
# From now on, we start to build our model
# We are also really big sad that we can't use maxpool inside sequential

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # image shape is 28 * 28 * 1, where 1 is one color channel
        # 28 * 28 is the image size
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        # output shape = (28 - 3 + 1) * (28 - 3 + 1) * 3 = 26 * 26 * 32
        # maxpooling
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        # output shape = 13 * 13 * 32
        # Note the volumn depth is not changed
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=4)
        # output shape = (13 - 4 + 1) * (13 - 4 + 1) * 64 = 10 * 10 * 16
        # maxpooling
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        # output shape = 10 * 10 * 16

        self.dropout = nn.Dropout(p=0.2)
        
        # fully connected layer
        self.fc1 = nn.Linear(10 * 10 * 16, 600)
        self.fc2 = nn.Linear(600, 120)
        self.fc3 = nn.Linear(120, 10)
        self.initialize_weights()
        
    def forward(self, x):
        # first conv
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.dropout(x)
        # flatten all dimensions except batch
        x = torch.flatten(x, 1)

        # fully connected layers
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ConvNet().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [6]:
model.train()
epoches = 75
lost = []
for epoch in tqdm(range(epoches)):
    train_loss, correct = 0, 0
    for X, y in train_dataloader:
        X, y = X.to(device), y.to(device)
        
        pred = model(X)

        loss = loss_fn(pred, y.long())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # record loss
        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    size = len(train_dataloader.dataset)
    train_loss /= len(train_dataloader)
    correct /= size
    print(f" Train accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f}")
    lost.append(train_loss)


  1%|▏         | 1/75 [00:05<06:54,  5.60s/it]

 Train accuracy: 10.0%, Avg loss:      nan


  3%|▎         | 2/75 [00:11<06:40,  5.48s/it]

 Train accuracy: 10.0%, Avg loss:      nan


  4%|▍         | 3/75 [00:16<06:33,  5.47s/it]

 Train accuracy: 10.0%, Avg loss:      nan
